In [1]:
library(data.table) 
library(stargazer)
library(lmtest)

set.seed(1)


Please cite as: 

 Hlavac, Marek (2015). stargazer: Well-Formatted Regression and Summary Statistics Tables.
 R package version 5.2. http://CRAN.R-project.org/package=stargazer 

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



# Estimating Heterogeneous Treatment Effects 
As we're discussing in the lecture, identifying parts of your data that react systematically differently to treatment is of interest for both theoretical and practical reasons. 

1. If we know that some people react differently to the treatment, we might be able to reason about "why" the treatment is working the way it does. (*Although as we're going to talk about later in the course, this is a pretty risky enterprise, at least if you have to make precise, evidence-based statements.*) 
2. If we know that some people react differently to treatment than others, we might want to *target* the treatment to reach certain populations (and not others). 

In this short demonstration, we show how to estimate heterogeneous treatment effects using linear models, as well as best practices for estimating and reporting in ways that will be both statistically sound, and also interpretable for your audience. 

## Make some data 

Create data, of size `N=1000`. In this data, 

- Roughly 1/2 the population are `female` 
- The population has seven `affluence` levels that we might think of as being ordered categories 
- A set of potential outcomes to control that have a mean of 100
- An individual response to treatment that follow this form: 
  - The mean response to treatment for poor men is 5; 
  - The mean response to treatment for poor women in 10; 
  - For both men and women, the higher is their affluence, the stronger is their response to treatment. 
  
*There are part of the population that react systematically differently to treatment than others*! Can we put an estimator together that will identify this? 

In [2]:
simSize <- 1000

d <- data.table(
    female = sample(c(0,1), simSize, replace = TRUE),
    affluence = sample(1:7, simSize, replace = TRUE),
    y0 = rnorm(simSize, mean = 100, sd = 7)
    ) 
d[ , ':='(tau   = rnorm(simSize, mean = 5 + 2*female + affluence), 
          treat = sample(0:1, simSize, replace = TRUE))]

Then, if people are in treatment, show their potential outcome to treatment (`y0+tau`); if they are in control, show their potential outcome to control. 

In [3]:
d[ , Y :=  y0 + treat * tau]

Use grouping and means to answer the following questions: 

- What should the mean of the treatment effects be? 
- Is the mean of `tau` higher for men or women? 
- Is the mean of `tau` higher for wealthy or poor indiviudals? 

Estimate a model, on the observable data (`Y`, `female`, `affluence`, and `treat`) that produces an estimate of the overall average treatment effect, and also improves the model performance by estimating the *non-causal* differences between men and women. 

In [4]:
overall_mod <- d[ , lm(Y ~ treat + female)]
summary(overall_mod)


Call:
lm(formula = Y ~ treat + female)

Residuals:
     Min       1Q   Median       3Q      Max 
-22.8764  -5.1092   0.0391   5.1297  22.9016 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  99.4286     0.4084 243.447   <2e-16 ***
treat         9.9370     0.4766  20.851   <2e-16 ***
female        0.9554     0.4763   2.006   0.0451 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 7.518 on 997 degrees of freedom
Multiple R-squared:  0.3077,	Adjusted R-squared:  0.3063 
F-statistic: 221.5 on 2 and 997 DF,  p-value: < 2.2e-16


# Estimate the Heterogeneous Treatment Effect
When we're estimating a heterogeneous treatment effect, we're effectively asking, "Is the treatment effect in *this group* different from the treatment effect in *that group*? 

The null hypothesis in this case is: 

> There isn't enough evidence to suggest that the treatment effects are different between the two groups, 

and so the alternative hypothesis is: 

> There is evidence that suggests *this group* and *that group* react differently to treatment. 

To estimate this HTE, we can include an additional model term, **an interaction between the treatment and group indicator** that will permit us to *directly* answer the questions we've just laid out. 

In the case of our data here, to evaluate whether there is a difference in the ways that men and women react to treatment we estimate a model of the following form: 

In [5]:
hte_mod = d[ , lm(Y ~ treat + female + treat * female)]
summary(hte_mod)$coefficients

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),100.2108010,0.4665088,214.810081,0.000000e+00
treat,8.3905055,0.6559695,12.790999,8.535074e-35
female,-0.7531087,0.6894760,-1.092291,2.749692e-01
treat:female,3.2372288,0.9490565,3.410997,6.732546e-04


What is being reported in this set of estimates: 

- The **(Intercept)** estimate is the mean value of `Y` for **men** who are in **control**; 
- The **treat** estimate is the mean treatment effect for **men**; 
- The **female** estimate is the mean difference between **men** and **women** who are in control;
- The **treat:female** estimate is the mean difference between the treatment effect for **men** and **women**. 

There is a lot to unpack in that set of statements. Once a model interacts some features with others, the interpretation of both will *always* be interwound. 

1. In this model, it is not possible to make a statement about the treatment effect, without also talking about the population, either men or women.
2. It is also not possible to talk about the average difference between men and women from these coefficients. Reported in this model is the average difference between men and women, *who are in the control group*. 

At the same time, although we made the data such that there is a heterogeneous response to treatment across levels of affluence, because we have not estimated this heterogeneity, the treatment effect is pooling across all levels of affluence. 

In the model below, we include an indicator for the affluence of the observation, *but we do not interact that indicator with the treatment indicator*. 

In [6]:
model_affluence = d[ , lm(Y ~ treat + female + factor(affluence) + treat * female)]
summary(model_affluence)$coefficients

,Estimate,Std. Error,t value,Pr(>|t|)
(Intercept),98.0745421,0.6957082,140.970791,0.000000e+00
treat,8.3610908,0.6474070,12.914736,2.240523e-35
female,-0.7573496,0.6798652,-1.113970,2.655624e-01
factor(affluence)2,1.2748043,0.8440001,1.510432,1.312524e-01
factor(affluence)3,0.9084284,0.8401050,1.081327,2.798149e-01
factor(affluence)4,2.5774230,0.8684264,2.967923,3.070344e-03
factor(affluence)5,3.1029397,0.8769886,3.538176,4.216054e-04
factor(affluence)6,3.4459593,0.8504190,4.052072,5.474135e-05
factor(affluence)7,4.2191340,0.8426264,5.007123,6.538388e-07
treat:female,3.2309811,0.9348958,3.455980,5.715777e-04


In this model, we continue to observe that 

- There is no baseline difference between men and women who are in control; 
- There is a treatment effect for men that is around 9 units; 
- The treatment effect for women is roughly 1.5 units stronger than for men.

We also see that there are differences in the potential outcomes to control *for men and women* that increase as measured affluence increases. In this model, because we have not interacted the `affluence` measure with the treatment, we are picking up only the **main effect** of affluence, not the heterogenity in treatment effects across affluence. 

## Reporting 

Reporting these heterogeneities to stakeholders is important, and poses particular challenges. Although interaction terms are reasonably straightforward to estimate and interpret, they do require more care than the simpler models. 

We prefer to report heterogeneous treatment effects as conditional effects *among parts of the data* when we're reporting to stakeholders, and support our conclusions of different responses to treatment with a test to back it up. 

Here's what we mean: 

1. Estimate the interaction model we have already show to test for HTE. 
2. When present, if possible partition the data into groups based on the HTE factor and estimate models. 
3. Report these models, and support conclusions of difference with the test from part 1. 

In [7]:
hte_mod    = d[ , lm(Y ~ treat * female)]
female_mod = d[female==1, lm(Y ~ treat)]
male_mod   = d[female==0, lm(Y ~ treat)]

stargazer(female_mod, male_mod, hte_mod, 
          type = 'text', omit.stat = c('ser', 'F'))


                   Dependent variable:      
             -------------------------------
                            Y               
                (1)       (2)        (3)    
--------------------------------------------
treat        11.628***  8.391***   8.391*** 
              (0.691)   (0.651)    (0.656)  
                                            
female                              -0.753  
                                   (0.689)  
                                            
treat:female                       3.237*** 
                                   (0.949)  
                                            
Constant     99.458*** 100.211*** 100.211***
              (0.511)   (0.463)    (0.467)  
                                            
--------------------------------------------
Observations    480       520       1,000   
R2             0.372     0.243      0.316   
Adjusted R2    0.371     0.241      0.314   
Note:            *p<0.1; **p<0.05; ***p<0.01


# A General Framework 
The framework that we've proposed to this point, estimating interactions and reporting, provide inferential statements on *individual parameter-level heterogeneity*. In the case of a female-male heterogeneity, the interaction tests exactly the contrast we seek to understand. 

However, if there are more than two levels in the feature that encodes the heterogeneity, coefficient-level tests will check for heterogeneity at each parameter-level -- and this might not answer the question we're actually interested in. 

In this section, we will (re-)introduce a general framework that produces the same inference as the *individual parameter-level tests* when there are only two levels, but that also generalizes to a multi-level variable -- the **F-test**. 

Above, in `overall_mod` you have estimated a model that forces treatment effects to be the same for men and women. In `hte_mod` you've permitted more model flexibility: namely that the treatment effects for men and women can be different. To test for whether the more flexible model has out-performed the less flexible model, recall in the F-test from 203. 

An F-test is a test for the ratio of variance of the residuals, $\epsilon$ from a regression. Specifically, if we have estimated two models: 

\begin{align}
Y_{i} & = \beta_{0} + \beta_{1} female + \beta_{2} treat + \epsilon_{1} \\ 
Y_{i} & = \beta_{0} + \beta_{1} female + \beta_{2} treat + \beta_{3} treat \times female + \epsilon_{2} 
\end{align} 

Then an F-test tests for the ratio of the variance of the $\epsilon$, which is known to follow an F-distribution. The null hypothesis in this test is that there is no difference in the distriutions, while a significant result on the test would provide evidence that one distribution has measurably more variance than the other. 

In [8]:
anova(overall_mod, hte_mod, test = 'F')

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
997,56358.07,NA,NA,NA,NA
996,55707.31,1,650.752,11.6349,0.0006732546


Note the p-value on the test here and compare it to the p-value that is reported on the interaction term in the `hte_mod` regression. 

In [9]:
coeftest(hte_mod)


t test of coefficients:

              Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)  100.21080    0.46651 214.8101 < 2.2e-16 ***
treat          8.39051    0.65597  12.7910 < 2.2e-16 ***
female        -0.75311    0.68948  -1.0923 0.2749692    
treat:female   3.23723    0.94906   3.4110 0.0006733 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


These two tests report *exactly* the same p-values. This is because when a single additional feature is being tested (as is the case with the single additional hte feature of `treat:female`) the two tests are mathmatically equivalent. 

**Why is this desirable?** The f-test provides a general framework that can test for heterogeneity across a feature, no matter the number of levels that exist in that feature. 

## Test for Heterogeneity on Affluence

The affluence feature that is in this data is *also* associated with increasing response to treatment -- that is to say -- there is heterogeneity of the treatment effect across levels of affluence. However, when we built the data, we said that the `affluence` level was a set of oredered categories; and so, we cannot use this feature as a linear predictor. 

Let's fit this model.

In [10]:
mod_short <- d[ , lm(Y ~ treat + female + factor(affluence))]
mod_long  <- d[ , lm(Y ~ treat + female + factor(affluence) + treat * factor(affluence))]

*(Notice that this regression is converting to a factor at the time of estimation)*. 

To test for whether there is heterogeneous response to treatment at different levels of affluence, we can use the F-test just described. 

In [11]:
anova(mod_short, mod_long, test = 'F')

Res.Df,RSS,Df,Sum of Sq,F,Pr(>F)
991,54271.55,NA,NA,NA,NA
985,53704.80,6,566.7516,1.732466,0.1102387


In the `Df` column, in the second row, we see that there are 6 fewer degrees of freedom in the long model than the short model -- this corresponds to an interacted coefficient for each of the levels that *are not* `affluence == 1`. The p-value for this test of overall heterogeneity across affluence suggests that, indeed, there is a difference in response. 

If we were to examine this at the individual *parameter-level* using only tests for the interactions, we would have reached a different conclusion. 

In [12]:
coeftest(mod_long)


t test of coefficients:

                         Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)              98.22518    0.82563 118.9694 < 2.2e-16 ***
treat                     7.97181    1.17155   6.8045 1.758e-11 ***
female                    0.93617    0.46951   1.9939  0.046434 *  
factor(affluence)2        0.71520    1.22467   0.5840  0.559358    
factor(affluence)3        0.57746    1.20880   0.4777  0.632963    
factor(affluence)4        1.84801    1.23636   1.4947  0.135307    
factor(affluence)5        1.90651    1.21914   1.5638  0.118184    
factor(affluence)6        2.00086    1.23054   1.6260  0.104268    
factor(affluence)7        1.84225    1.17955   1.5618  0.118651    
treat:factor(affluence)2  1.23373    1.70145   0.7251  0.468558    
treat:factor(affluence)3  0.86789    1.68940   0.5137  0.607558    
treat:factor(affluence)4  1.42304    1.74583   0.8151  0.415208    
treat:factor(affluence)5  2.54583    1.76128   1.4454  0.148652    
treat:factor(affluence

In this set of tests, there is not enough evidence to reject the null hypothesis that the treatment works differently for people who are affluence level 1 than people who are affluence level **2, 3, 4, 5**, while there is some evidence that levels **6, and 7** respond differently to treatment than level **1** people. 